In [1]:
import pandas as pd
import pandas_ta as ta
import importlib
import os
import sys
import yfinance as yf

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

Input needed:

1. prices.csv
-date, asset, Close, Expected Return (from AR model), other components needed to run strategies

2. strategy_map.csv
- asset, strategy (e.g., Strategy05), ideal_proportion

Config Example:

config = {
    "starting_cash": 1000000,        # Initial capital
    "buy_pct": 0.05,               # Each buy = 5% of max cap
    "cash_floor_pct": 0.10,        # Minimum 10% cash must be held
    "cash_ceiling_pct": 0.30,      # If cash > 30%, consider buying gold
    "fee": 0.001,                  # Transaction fee (0.1%)
    "tp_pct": 0.10,                # Take-profit threshold (10%) 
    "sl_pct": 0.05                 # Stop-loss threshold (5%)
}

Notes: the tp, sl pct gonna make "dynamic" - different for each asset.

In [ ]:
# Create some sample data to try now
config = {
    "starting_cash": 1000000,  # Initial capital
    "buy_pct": 0.1,  # Each buy = 5% of max cap
    "cash_floor_pct": 0.10,  # Minimum 10% cash must be held
    "tp_pct": 0.1,  # Take-profit threshold (10%)
    "sl_pct": 0.05,  # Stop-loss threshold (5%)
}

classname_to_filename = {
    "BollingerBandsBreakout": "bollinger_bands_breakout",
    "LarryWilliamsPriceAction": "larry_williams_price_action",
    "MACDBollingerBandsMeanReversion": "macd_bollinger_bands_mean_reversion",
    "MeanReversion": "mean_reversion",
    "MichaelHarrisPriceAction": "michael_harris_price_action",
    "Momentum": "momentum",
    "RSIDivergence": "rsi_divergence",
    "Scalping": "scalping",
    "VolumeSpikeReversal": "volume_spike_reversal",
}

strategy_map = pd.read_csv("../data/processed/asset_strategies.csv").set_index("asset")
strategy_map

,weight,strategy,return
asset,,,
CZR,0.195536,Scalping,4.840157
INTC,0.159487,MeanReversion,29.333193
MHK,0.115818,MeanReversion,14.363444
BLDR,0.107738,RSIDivergence,25.424814
URI,0.088408,MichaelHarrisPriceAction,15.062324
ON,0.070599,RSIDivergence,36.768206
NCLH,0.031879,LarryWilliamsPriceAction,64.092653
ALB,0.028309,RSIDivergence,41.214243
VST,0.005340,LarryWilliamsPriceAction,113.479470


First function retrieves data from yf 

In [ ]:
def get_asset_df(strategy_map, start, end):
    df_list = []
    for asset_name in strategy_map.index:
        # Retrieve yf finance data
        df = yf.Ticker(asset_name).history(start=start, end=end, actions=False)

        # Align date formatting
        df = df.reset_index()
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce").dt.strftime("%Y-%m-%d")

        df["Asset"] = asset_name
        df_list.append(df.reset_index())

    df_all = pd.concat(df_list).set_index(["Date", "Asset"]).sort_index()
    return df_all

In [ ]:
asset_df = get_asset_df(strategy_map=strategy_map, start="2023-01-01", end="2025-03-01")

asset_df.head(10)

index        Open        High         Low       Close  \
Date       Asset                                                             
2023-01-01 AVAX-USD      0   10.903734   10.929948   10.670733   10.865915   
           LINK-USD      0    5.568981    5.628169    5.517975    5.622443   
           SOL-USD       0    9.961036   10.052801    9.721011    9.982173   
2023-01-02 AVAX-USD      1   10.866034   11.231228   10.743951   11.153615   
           LINK-USD      1    5.622763    5.737175    5.567815    5.687627   
           SOL-USD       1    9.983222   11.372013    9.845211   11.272967   
2023-01-03 ALB           0  212.742399  213.062752  202.471754  208.228363   
           AVAX-USD      2   11.153684   11.528151   11.087236   11.384450   
           BLDR          0   65.949997   66.570000   64.540001   65.349998   
           CZR           0   42.930000   43.290001   41.410000   42.259998   

                        Volume  
Date       Asset                
2023-01-01 AVAX-USD   95741904  
           LINK-USD  109175362  
           SOL-USD   194221164  
2023-01-02 AVAX-USD  128530323  
           LINK-USD  179768004  
           SOL-USD   558570124  
2023-01-03 ALB         2293400  
           AVAX-USD  165410541  
           BLDR        2194900  
           CZR         3007400

Second function implements AR model to derive expected return 

In [ ]:
def compute_returns(asset_df):
    asset_df["Return"] = asset_df.groupby("Asset")["Close"].pct_change()
    return asset_df


asset_df_with_returns = compute_returns(asset_df)

In [6]:
asset_df_with_returns.head(10)

index        Open        High         Low       Close  \
Date       Asset                                                             
2023-01-01 AVAX-USD      0   10.903734   10.929948   10.670733   10.865915   
           LINK-USD      0    5.568981    5.628169    5.517975    5.622443   
           SOL-USD       0    9.961036   10.052801    9.721011    9.982173   
2023-01-02 AVAX-USD      1   10.866034   11.231228   10.743951   11.153615   
           LINK-USD      1    5.622763    5.737175    5.567815    5.687627   
           SOL-USD       1    9.983222   11.372013    9.845211   11.272967   
2023-01-03 ALB           0  212.742399  213.062752  202.471754  208.228363   
           AVAX-USD      2   11.153684   11.528151   11.087236   11.384450   
           BLDR          0   65.949997   66.570000   64.540001   65.349998   
           CZR           0   42.930000   43.290001   41.410000   42.259998   

                        Volume    Return  
Date       Asset                          
2023-01-01 AVAX-USD   95741904       NaN  
           LINK-USD  109175362       NaN  
           SOL-USD   194221164       NaN  
2023-01-02 AVAX-USD  128530323  0.026477  
           LINK-USD  179768004  0.011593  
           SOL-USD   558570124  0.129310  
2023-01-03 ALB         2293400       NaN  
           AVAX-USD  165410541  0.020696  
           BLDR        2194900       NaN  
           CZR         3007400       NaN

In [7]:
def generate_expected_returns(data, ar_results):
    """ Calculate expected returns using fitted AR models """
    new_data = data.reset_index().merge(ar_results, on="Asset", how="left").set_index(["Date", "Asset"])
    new_data["Expected Return"] = new_data["const"] + new_data["Return"] * new_data["ar.L1"]
    return new_data

In [8]:
ar_model_info = pd.read_csv("../data/processed/ar_model.csv")
ar_model_info

,Asset,Best Model,BIC,const,ar.L1,sigma2
0,CZR,"(1, 0, 0)",-2137.099847,-0.001004,-0.037306,0.000487
1,INTC,"(1, 0, 0)",-1898.767394,-0.001165,0.053588,0.000827
2,MHK,"(1, 0, 0)",-2237.419365,0.000454,0.064773,0.000389
3,BLDR,"(1, 0, 0)",-2146.278473,-0.000304,-0.037379,0.000477
4,URI,"(1, 0, 0)",-2284.821016,0.000504,-0.071571,0.000350
5,ON,"(1, 0, 0)",-2053.295520,-0.001143,-0.153363,0.000586
6,NCLH,"(1, 0, 0)",-2008.360786,0.000543,-0.014170,0.000648
7,ALB,"(1, 0, 0)",-1908.997544,-0.000997,-0.151407,0.000808
8,VST,"(1, 0, 0)",-1739.133780,0.003166,-0.093050,0.001179
9,AVAX-USD,"(1, 0, 0)",-1402.471749,-0.000203,0.003612,0.002473


In [ ]:
# Apply AR model
asset_df_with_AR = generate_expected_returns(asset_df_with_returns, ar_model_info)
asset_df_with_AR = asset_df_with_AR.drop(['Best Model','BIC', 'const','ar.L1', 'sigma2'], axis=1)
asset_df_with_AR

index        Open        High         Low       Close  \
Date       Asset                                                             
2023-01-01 AVAX-USD      0   10.903734   10.929948   10.670733   10.865915   
           LINK-USD      0    5.568981    5.628169    5.517975    5.622443   
           SOL-USD       0    9.961036   10.052801    9.721011    9.982173   
2023-01-02 AVAX-USD      1   10.866034   11.231228   10.743951   11.153615   
           LINK-USD      1    5.622763    5.737175    5.567815    5.687627   
...                    ...         ...         ...         ...         ...   
2025-02-28 NCLH        540   23.540001   23.680000   22.280001   22.719999   
           ON          540   47.939999   48.520000   46.040001   47.049999   
           SOL-USD     789  137.620773  148.183838  125.742218  148.030014   
           URI         540  635.500000  643.719971  627.760010  642.320007   
           VST         540  127.779188  134.477620  123.836006  133.429428   

                         Volume    Return  Expected Return  
Date       Asset                                            
2023-01-01 AVAX-USD    95741904       NaN              NaN  
           LINK-USD   109175362       NaN              NaN  
           SOL-USD    194221164       NaN              NaN  
2023-01-02 AVAX-USD   128530323  0.026477        -0.000108  
           LINK-USD   179768004  0.011593         0.001155  
...                         ...       ...              ...  
2025-02-28 NCLH        24852400 -0.042159         0.001140  
           ON          13110600 -0.006965        -0.000075  
           SOL-USD   7544673033  0.075637        -0.004608  
           URI           694800  0.014451        -0.000530  
           VST         13060900  0.028075         0.000554  

[7239 rows x 8 columns]

In [ ]:
def calculate_technical_indicators(df):
    df = df.copy()

    # Core indicators
    df["EMA_50"] = ta.ema(df["Close"], length=50)
    df["EMA_200"] = ta.ema(df["Close"], length=200)
    df["RSI"] = ta.rsi(df["Close"], length=14)
    df["ATR"] = ta.atr(df["High"], df["Low"], df["Close"], length=7)

    # Bollinger Bands
    bbands = ta.bbands(df["Close"], length=20)
    bbands = bbands.rename(
        columns={
            "BBU_20_2.0": "Upper_Band",
            "BBM_20_2.0": "Middle_Band",
            "BBL_20_2.0": "Lower_Band",
            "BBB_20_2.0": "Band_Width",
            "BBP_20_2.0": "Percent_B",
        }
    )

    # MACD
    macd = ta.macd(df["Close"])
    macd = macd.rename(
        columns={
            "MACD_12_26_9": "MACD",
            "MACDh_12_26_9": "Histogram",
            "MACDs_12_26_9": "Signal",
        }
    )

    # Join indicator DataFrames
    df = df.join([bbands, macd])

    return df

In [11]:
enriched_df = calculate_technical_indicators(asset_df_with_AR)
enriched_df

index        Open        High         Low       Close  \
Date       Asset                                                             
2023-01-01 AVAX-USD      0   10.903734   10.929948   10.670733   10.865915   
           LINK-USD      0    5.568981    5.628169    5.517975    5.622443   
           SOL-USD       0    9.961036   10.052801    9.721011    9.982173   
2023-01-02 AVAX-USD      1   10.866034   11.231228   10.743951   11.153615   
           LINK-USD      1    5.622763    5.737175    5.567815    5.687627   
...                    ...         ...         ...         ...         ...   
2025-02-28 NCLH        540   23.540001   23.680000   22.280001   22.719999   
           ON          540   47.939999   48.520000   46.040001   47.049999   
           SOL-USD     789  137.620773  148.183838  125.742218  148.030014   
           URI         540  635.500000  643.719971  627.760010  642.320007   
           VST         540  127.779188  134.477620  123.836006  133.429428   

                         Volume    Return  Expected Return      EMA_50  \
Date       Asset                                                         
2023-01-01 AVAX-USD    95741904       NaN              NaN         NaN   
           LINK-USD   109175362       NaN              NaN         NaN   
           SOL-USD    194221164       NaN              NaN         NaN   
2023-01-02 AVAX-USD   128530323  0.026477        -0.000108         NaN   
           LINK-USD   179768004  0.011593         0.001155         NaN   
...                         ...       ...              ...         ...   
2025-02-28 NCLH        24852400 -0.042159         0.001140  108.488325   
           ON          13110600 -0.006965        -0.000075  106.078979   
           SOL-USD   7544673033  0.075637        -0.004608  107.724118   
           URI           694800  0.014451        -0.000530  128.688662   
           VST         13060900  0.028075         0.000554  128.874575   

                        EMA_200        RSI         ATR  Lower_Band  \
Date       Asset                                                     
2023-01-01 AVAX-USD         NaN        NaN         NaN         NaN   
           LINK-USD         NaN        NaN         NaN         NaN   
           SOL-USD          NaN        NaN         NaN         NaN   
2023-01-02 AVAX-USD         NaN        NaN         NaN         NaN   
           LINK-USD         NaN        NaN         NaN         NaN   
...                         ...        ...         ...         ...   
2025-02-28 NCLH      120.707891  47.562774  110.642259 -172.761572   
           ON        119.974976  48.346231   98.521937 -175.020849   
           SOL-USD   120.254131  51.579785   98.895065 -167.912598   
           URI       125.448816  63.593723  155.580050 -233.780527   
           VST       125.528225  49.873633  207.423472 -226.428182   

                     Middle_Band  Upper_Band  Band_Width  Percent_B  \
Date       Asset                                                      
2023-01-01 AVAX-USD          NaN         NaN         NaN        NaN   
           LINK-USD          NaN         NaN         NaN        NaN   
           SOL-USD           NaN         NaN         NaN        NaN   
2023-01-02 AVAX-USD          NaN         NaN         NaN        NaN   
           LINK-USD          NaN         NaN         NaN        NaN   
...                          ...         ...         ...        ...   
2025-02-28 NCLH        92.489464  357.740501  573.581084   0.368484   
           ON          90.915552  356.851953  585.018505   0.417526   
           SOL-USD     97.192399  362.297396  545.526193   0.595882   
           URI        122.350900  478.482326  582.147622   1.230024   
           VST        127.359871  481.147924  555.572254   0.508578   

                          MACD  Histogram    Signal  
Date       Asset                                     
2023-01-01 AVAX-USD        NaN        NaN       NaN  
           LINK-USD        NaN        NaN     

Third function to add signal from strategies 

In [12]:
def load_data_and_apply_strategies(df_all, strategy_map):
    df_list = []

    for asset_name in strategy_map.index:
        try:
            # Get strategy module + class
            strat_class_name = strategy_map.loc[asset_name, "strategy"]
            strat_file_name = classname_to_filename[strat_class_name]

            strategy_module = importlib.import_module(
                f"strategies.custom.{strat_file_name}"
            )
            strategy_class = getattr(strategy_module, strat_class_name)
            strategy_instance = strategy_class()

            # Get asset's DataFrame slice
            asset_df = df_all.xs(asset_name, level="Asset").copy()

            # Apply strategy
            asset_df = strategy_instance.generate_signals(asset_df)

            # Restore asset label if removed
            asset_df["Asset"] = asset_name
            df_list.append(asset_df.reset_index())

        except Exception as e:
            print(f"Error processing {asset_name} ({strat_class_name}): {e}")
            continue

    # Combine and restore multi-index
    df_with_signals = pd.concat(df_list).set_index(["Date", "Asset"]).sort_index()
    return df_with_signals

In [13]:
df_with_signals = load_data_and_apply_strategies(enriched_df, strategy_map)

In [ ]:
valid_df_with_signals = df_with_signals.loc[df_with_signals.index.get_level_values('Date') >= '2025-01-01']
valid_df_with_signals

level_0  index        Open        High         Low  \
Date       Asset                                                          
2025-01-01 AVAX-USD      NaN    731   35.690929   37.869911   34.960541   
           LINK-USD      NaN    731   20.001163   21.771200   19.707378   
           SOL-USD       NaN    731  189.266922  194.818497  187.879456   
2025-01-02 ALB           NaN    502   85.956465   87.497790   83.937822   
           AVAX-USD      NaN    732   37.693447   40.501595   37.689964   
...                      ...    ...         ...         ...         ...   
2025-02-28 NCLH          NaN    540   23.540001   23.680000   22.280001   
           ON            NaN    540   47.939999   48.520000   46.040001   
           SOL-USD       NaN    789  137.620773  148.183838  125.742218   
           URI           NaN    540  635.500000  643.719971  627.760010   
           VST           NaN    540  127.779188  134.477620  123.836006   

                          Close      Volume    Return  Expected Return  \
Date       Asset                                                         
2025-01-01 AVAX-USD   37.693439   334807162  0.056107    -6.013646e-07   
           LINK-USD   21.674845   670017230  0.083668     1.535036e-03   
           SOL-USD   193.873734  2324231668  0.024354    -5.260706e-04   
2025-01-02 ALB        84.753235     1866100 -0.009875     4.978192e-04   
           AVAX-USD   39.229160   522067297  0.040742    -5.610548e-05   
...                         ...         ...       ...              ...   
2025-02-28 NCLH       22.719999    24852400 -0.042159     1.140214e-03   
           ON         47.049999    13110600 -0.006965    -7.505876e-05   
           SOL-USD   148.030014  7544673033  0.075637    -4.607648e-03   
           URI       642.320007      694800  0.014451    -5.301611e-04   
           VST       133.429428    13060900  0.028075     5.537452e-04   

                         EMA_50  ...  Middle_Band  Upper_Band  Band_Width  \
Date       Asset                 ...                                        
2025-01-01 AVAX-USD  134.809535  ...   143.823889  533.029612  541.225421   
           LINK-USD  130.372880  ...   143.878457  533.015349  540.924472   
           SOL-USD   132.863110  ...   147.645144  537.186242  527.672077   
2025-01-02 ALB       130.976448  ...   150.595306  537.282319  513.544579   
           AVAX-USD  127.378515  ...   149.383764  537.305079  519.362086   
...                         ...  ...          ...         ...         ...   
2025-02-28 NCLH      108.488325  ...    92.489464  357.740501  573.581084   
           ON        106.078979  ...    90.915552  356.851953  585.018505   
           SOL-USD   107.724118  ...    97.192399  362.297396  545.526193   
           URI       128.688662  ...   122.350900  478.482326  582.147622   
           VST       128.874575  ...   127.359871  481.147924  555.572254   

                     Percent_B       MACD  Histogram     Signal  EMASignal  \
Date       Asset                                                             
2025-01-01 AVAX-USD   0.363658  18.767103   8.649790  10.117313        NaN   
           LINK-USD   0.342981   6.163310  -3.163203   9.326513        NaN   
           SOL-USD    0.559337   9.954970   0.502766   9.452204        NaN   
2025-01-02 ALB        0.414864   4.107428  -4.275821   8.383249        NaN   
           AVAX-USD   0.358019  -4.152338 -10.028470   5.876132        NaN   
...                        ...        ...        ...        ...        ...   
2025-02-28 NCLH       0.368484 -17.470936 -10.763449  -6.707487        NaN   
           ON         0.417526 -19.031284  -9.859038  -9.172246        NaN   
           SOL-USD    0.595882 -11.981513  -2.247413  -9.734100        NaN   
           URI        1.230024  33.108894  34.274395  -1.165501        NaN   
           VST        0.508578  27.463567  22.903254   4.560313        NaN   

                     TotalSignal   AvgVolume10  
Date       Asset

In [15]:
def calculate_fee(amount):
    return max(0.00025 * amount, 0.50)

In [16]:
def auto_trade(df, strategy_map, config):
    if not isinstance(df.index, pd.MultiIndex):
        df = df.set_index(["date", "asset"])

    assets = df.index.get_level_values(1).unique()
    print(assets)
    dates = df.index.get_level_values(0).unique()

    # Initialize portfolio and cash
    portfolio = {asset: {"units": 0, "entry_price": None} for asset in assets}
    cash = config["starting_cash"]
    trade_log = []

    for date in dates:
        # Calculate total portfolio value at start of day
        total_value = cash + sum(
            portfolio[asset]["units"] * df.loc[(date, asset), "Close"]
            for asset in assets
            if (date, asset) in df.index
        )

        # Calculate thresholds
        cash_floor = config["cash_floor_pct"] * total_value
        # cash_ceiling = config['cash_ceiling_pct'] * total_value

        for asset in assets:
            if (date, asset) not in df.index:
                continue
            row = df.loc[(date, asset)]
            current_price = row["Close"]
            signal = row["TotalSignal"]
            # asset_type = strategy_map.loc[asset, 'type']
            # asset_type = strategy_map.loc[strategy_map['asset'] == asset, 'type'].values[0]

            # Get max allocation for this asset
            ideal_proportion = strategy_map.loc[asset, "weight"]
            # ideal_proportion = strategy_map.loc[strategy_map['asset'] == asset, 'weight'].values[0]
            max_asset_value = ideal_proportion * total_value
            current_allocation = portfolio[asset]["units"] * current_price
            remaining_allocation = max_asset_value - current_allocation
            buy_chunk = config["buy_pct"] * max_asset_value
            buy_amount = min(buy_chunk, remaining_allocation)

            # ---------- SELL CONDITIONS ----------
            # TP/SL check
            if portfolio[asset]["units"] > 0:
                entry_price = portfolio[asset]["entry_price"]
                profit_pct = (current_price - entry_price) / entry_price
                if (
                    profit_pct >= config["tp_pct"] or profit_pct <= -config["sl_pct"]
                ):  # IMPLEMENT DYNAMIC TP AND SL
                    if asset.endswith("-USD"):  # Crypto
                        sell_price = current_price * 0.99  # apply 1% spread
                        proceeds = portfolio[asset]["units"] * sell_price
                    else:
                        fee = calculate_fee(portfolio[asset]["units"] * current_price)
                        proceeds = portfolio[asset]["units"] * current_price - fee
                    cash += proceeds
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": (
                                "SELL_TP"
                                if profit_pct >= config["tp_pct"]
                                else "SELL_SL"
                            ),
                            "price": current_price,
                            "units": portfolio[asset]["units"],
                        }
                    )
                    portfolio[asset] = {"units": 0, "entry_price": None}
                    continue  # skip further actions after selling

            # Strategy signal: SHORT → sell everything
            if signal == 1 and portfolio[asset]["units"] > 0:
                if asset.endswith("-USD"):  # Crypto
                    sell_price = current_price * 0.99  # apply 1% spread
                    proceeds = portfolio[asset]["units"] * sell_price
                else:
                    fee = calculate_fee(portfolio[asset]["units"] * current_price)
                    proceeds = portfolio[asset]["units"] * current_price - fee
                cash += proceeds
                trade_log.append(
                    {
                        "asset": asset,
                        "date": date,
                        "action": "SELL_SHORT",
                        "price": current_price,
                        "units": portfolio[asset]["units"],
                    }
                )
                portfolio[asset] = {"units": 0, "entry_price": None}
                continue

            # ---------- BUY CONDITIONS ----------
            if signal == 2:
                expected_return = row.get("Expected Return", None)
                baseline_fee_pct = 0.00025
                if expected_return is None or expected_return <= 2 * baseline_fee_pct:
                    continue  # don't buy if expected return is missing or too low

                # Check if enough cash remains after purchase
                if buy_amount > 0 and (cash - buy_amount) >= cash_floor:
                    if asset.endswith("-USD"):  # Crypto
                        buy_price = current_price * 1.01  # apply 1% spread
                        new_units = buy_amount / buy_price
                    else:
                        fee = calculate_fee(buy_amount)
                        new_units = (buy_amount - fee) / current_price

                    old_units = portfolio[asset]["units"]
                    old_price = portfolio[asset]["entry_price"]

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * current_price)
                        / total_units
                        if old_units > 0
                        else current_price
                    )  # update price in portfolio so that we can calculate the profit/ loss accurately

                    # Update portfolio
                    portfolio[asset]["units"] = total_units
                    portfolio[asset]["entry_price"] = new_avg_price
                    cash -= buy_amount

                    # Log trade
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": "BUY_ADD" if old_units > 0 else "BUY_NEW",
                            "price": current_price,
                            "units": new_units,
                        }
                    )

    # ---------- FINAL SUMMARY ----------
    final_value = cash + sum(
        portfolio[asset]["units"] * df.loc[(dates[-1], asset), "Close"]
        for asset in assets
        if (dates[-1], asset) in df.index
    )

    total_profit = final_value - config["starting_cash"]

    print("\nFinal Portfolio Value:", round(final_value, 2))
    print("Total Profit:", round(total_profit, 2))
    print("Number of Trades:", len(trade_log))

    trade_df = pd.DataFrame(trade_log)

    if not trade_df.empty:
        # Path
        path = "../data/misc/trade_log.csv"
        # Export to CSV
        trade_df.to_csv(path, index=False)
        print("Trade log exported to 'trade_log.csv'")

    else:
        print("\n(No trades were executed during the period.)")

    return trade_df, final_value

In [17]:
auto_trade(valid_df_with_signals, strategy_map, config)

Index(['AVAX-USD', 'LINK-USD', 'SOL-USD', 'ALB', 'BLDR', 'CZR', 'INTC', 'MHK',
       'NCLH', 'ON', 'URI', 'VST'],
      dtype='object', name='Asset')

Final Portfolio Value: 1001509.67
Total Profit: 1509.67
Number of Trades: 30
Trade log exported to 'trade_log.csv'


(       asset        date      action       price        units
 0         ON  2025-01-07     BUY_NEW   62.730000   112.515445
 1        ALB  2025-01-08     BUY_NEW   86.891205    32.555920
 2         ON  2025-01-08     SELL_SL   58.310001   112.515445
 3       BLDR  2025-01-10     BUY_NEW  139.960007    76.917642
 4       BLDR  2025-01-14     SELL_TP  154.250000    76.917642
 5        MHK  2025-01-14     BUY_NEW  125.580002    92.273694
 6        MHK  2025-01-15     BUY_ADD  130.059998    89.138814
 7   LINK-USD  2025-01-17     BUY_NEW   25.112717    68.326166
 8        ALB  2025-01-17     SELL_TP   96.944649    32.555920
 9       INTC  2025-01-17     BUY_NEW   21.490000   742.836983
 10       MHK  2025-01-17     BUY_ADD  129.270004    89.677247
 11       ALB  2025-01-21     BUY_NEW   93.523895    30.317995
 12       MHK  2025-01-21     BUY_ADD  130.660004    88.781830
 13       ALB  2025-01-22     BUY_ADD   89.745148    31.589663
 14       MHK  2025-01-23     BUY_ADD  130.830002    88

In [ ]:
def auto_trade_v2(df, strategy_map, config):
    if not isinstance(df.index, pd.MultiIndex):
        df = df.set_index(["date", "asset"])

    assets = df.index.get_level_values(1).unique()
    print(assets)
    dates = df.index.get_level_values(0).unique()

    # Initialize portfolio and cash
    portfolio = {asset: {"units": 0, "entry_price": None} for asset in assets}
    cash = config["starting_cash"]
    trade_log = []

    for date in dates:
        # Calculate total portfolio value at start of day
        total_value = cash + sum(
            portfolio[asset]["units"] * df.loc[(date, asset), "Close"]
            for asset in assets
            if (date, asset) in df.index
        )

        # Calculate thresholds
        cash_floor = config["cash_floor_pct"] * total_value
        # cash_ceiling = config['cash_ceiling_pct'] * total_value

        for asset in assets:
            if (date, asset) not in df.index:
                continue
            row = df.loc[(date, asset)]
            current_price = row["Close"]
            signal = row["TotalSignal"]
            # asset_type = strategy_map.loc[asset, 'type']
            # asset_type = strategy_map.loc[strategy_map['asset'] == asset, 'type'].values[0]

            # Get max allocation for this asset
            ideal_proportion = strategy_map.loc[asset, "weight"]
            # ideal_proportion = strategy_map.loc[strategy_map['asset'] == asset, 'weight'].values[0]
            max_asset_value = ideal_proportion * total_value
            current_allocation = portfolio[asset]["units"] * current_price
            remaining_allocation = max_asset_value - current_allocation
            buy_chunk = config["buy_pct"] * max_asset_value
            buy_amount = min(buy_chunk, remaining_allocation)

            # ---------- SELL CONDITIONS ----------
            # TP/SL check
            if portfolio[asset]["units"] > 0:
                entry_price = portfolio[asset]["entry_price"]
                profit_pct = (current_price - entry_price) / entry_price
                if (
                    profit_pct >= config["tp_pct"] or profit_pct <= -config["sl_pct"]
                ):  # IMPLEMENT DYNAMIC TP AND SL
                    if asset.endswith("-USD"):  # Crypto
                        sell_price = current_price * 0.99  # apply 1% spread
                        proceeds = portfolio[asset]["units"] * sell_price
                    else:
                        fee = calculate_fee(portfolio[asset]["units"] * current_price)
                        proceeds = portfolio[asset]["units"] * current_price - fee
                    cash += proceeds
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": (
                                "SELL_TP"
                                if profit_pct >= config["tp_pct"]
                                else "SELL_SL"
                            ),
                            "price": current_price,
                            "units": portfolio[asset]["units"],
                        }
                    )
                    portfolio[asset] = {"units": 0, "entry_price": None}
                    continue  # skip further actions after selling

            # Strategy signal: SHORT → sell everything
            if signal == 1 and portfolio[asset]["units"] > 0:
                if asset.endswith("-USD"):  # Crypto
                    sell_price = current_price * 0.99  # apply 1% spread
                    proceeds = portfolio[asset]["units"] * sell_price
                else:
                    fee = calculate_fee(portfolio[asset]["units"] * current_price)
                    proceeds = portfolio[asset]["units"] * current_price - fee
                cash += proceeds
                trade_log.append(
                    {
                        "asset": asset,
                        "date": date,
                        "action": "SELL_SHORT",
                        "price": current_price,
                        "units": portfolio[asset]["units"],
                    }
                )
                portfolio[asset] = {"units": 0, "entry_price": None}
                continue

            # ---------- BUY CONDITIONS ----------
            if signal == 2:
                expected_return = row.get("Expected Return", None)
                baseline_fee_pct = 0.00025
                if expected_return is None or expected_return <= 2 * baseline_fee_pct:
                    continue  # don't buy if expected return is missing or too low

                # Check if enough cash remains after purchase
                if buy_amount > 0 and (cash - buy_amount) >= cash_floor:
                    if asset.endswith("-USD"):  # Crypto
                        buy_price = current_price * 1.01  # apply 1% spread
                        new_units = buy_amount / buy_price
                    else:
                        fee = calculate_fee(buy_amount)
                        new_units = (buy_amount - fee) / current_price

                    old_units = portfolio[asset]["units"]
                    old_price = portfolio[asset]["entry_price"]

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * current_price)
                        / total_units
                        if old_units > 0
                        else current_price
                    )  # update price in portfolio so that we can calculate the profit/ loss accurately

                    # Update portfolio
                    portfolio[asset]["units"] = total_units
                    portfolio[asset]["entry_price"] = new_avg_price
                    cash -= buy_amount

                    # Log trade
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": "BUY_ADD" if old_units > 0 else "BUY_NEW",
                            "price": current_price,
                            "units": new_units,
                        }
                    )

    # ---------- FINAL SUMMARY ----------
    final_value = cash + sum(
        portfolio[asset]["units"] * df.loc[(dates[-1], asset), "Close"]
        for asset in assets
        if (dates[-1], asset) in df.index
    )

    total_profit = final_value - config["starting_cash"]

    print("\nFinal Portfolio Value:", round(final_value, 2))
    print("Total Profit:", round(total_profit, 2))
    print("Number of Trades:", len(trade_log))

    trade_df = pd.DataFrame(trade_log)

    if not trade_df.empty:
        # Add a signed P&L for each trade
        trade_df["signed_value"] = trade_df["price"] * trade_df["units"]
        trade_df["signed_value"] = trade_df.apply(
            lambda row: (
                -row["signed_value"]
                if row["action"].startswith("BUY")
                else row["signed_value"]
            ),
            axis=1,
        )

        # Net P&L by asset
        pnl_per_asset = trade_df.groupby("asset")["signed_value"].sum().sort_values()

        print("\nNet P&L by Asset (lowest to highest):")
        print(pnl_per_asset)

        worst_asset = pnl_per_asset.idxmin()
        worst_loss = pnl_per_asset.min()

        print(
            f"\nAsset that pulled down profit the most: {worst_asset} (${round(worst_loss, 2)})"
        )

    else:
        print("\n(No trades were executed during the period.)")

    return trade_df, final_value

In [19]:
auto_trade_v2(valid_df_with_signals, strategy_map, config)

Index(['AVAX-USD', 'LINK-USD', 'SOL-USD', 'ALB', 'BLDR', 'CZR', 'INTC', 'MHK',
       'NCLH', 'ON', 'URI', 'VST'],
      dtype='object', name='Asset')

Final Portfolio Value: 1001509.67
Total Profit: 1509.67
Number of Trades: 30

Net P&L by Asset (lowest to highest):
asset
NCLH       -3566.767282
MHK        -3117.740764
VST         -586.781334
ON          -497.318063
LINK-USD    -170.591263
ALB          122.876421
BLDR        1099.152592
INTC        4725.746648
Name: signed_value, dtype: float64

Asset that pulled down profit the most: NCLH ($-3566.77)


(       asset        date      action       price        units  signed_value
 0         ON  2025-01-07     BUY_NEW   62.730000   112.515445  -7058.093838
 1        ALB  2025-01-08     BUY_NEW   86.891205    32.555920  -2828.823123
 2         ON  2025-01-08     SELL_SL   58.310001   112.515445   6560.775775
 3       BLDR  2025-01-10     BUY_NEW  139.960007    76.917642 -10765.393727
 4       BLDR  2025-01-14     SELL_TP  154.250000    76.917642  11864.546319
 5        MHK  2025-01-14     BUY_NEW  125.580002    92.273694 -11587.730671
 6        MHK  2025-01-15     BUY_ADD  130.059998    89.138814 -11593.393947
 7   LINK-USD  2025-01-17     BUY_NEW   25.112717    68.326166  -1715.855659
 8        ALB  2025-01-17     SELL_TP   96.944649    32.555920   3156.122239
 9       INTC  2025-01-17     BUY_NEW   21.490000   742.836983 -15963.566591
 10       MHK  2025-01-17     BUY_ADD  129.270004    89.677247 -11592.578109
 11       ALB  2025-01-21     BUY_NEW   93.523895    30.317995  -2835.457007